# Web Scraping

In [18]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from selenium import webdriver
from time import sleep

In [19]:
driver_options = webdriver.ChromeOptions()
driver_options.add_argument('headless')
driver = webdriver.Chrome(options=driver_options)
driver.get('https://en.wikipedia.org/wiki/Category:Films_set_in_the_United_States_by_state')

sleep(2)
to_click = driver.find_elements_by_xpath('//*[@title="expand"]')
sleep(2)
while len(to_click) > 1:
    for element in to_click[1:]:
        element.click()
        sleep(2)
    to_click = driver.find_elements_by_xpath('//*[@title="expand"]')
    sleep(2)

soup = BeautifulSoup(driver.execute_script('return document.body.innerHTML'), 'html.parser')

In [20]:
state_links = []
for group in soup.find_all('div', class_='mw-category-group')[1:]:
    for state in group.find_all('li'):
        item = state.find('a')
        name = item.get_text(strip=True).split('in ')[-1].split(' (')[0]
        state_links.append((name, f"https://en.wikipedia.org{item['href']}"))
        for item in state.find('div', class_='CategoryTreeChildren').find_all('a'):
            state_links.append((name, f"https://en.wikipedia.org{item['href']}"))
state_links = [i for n, i in enumerate(state_links) if i not in state_links[:n]]

In [23]:
movie_list = []
for entry in state_links:
    state_name = entry[0]
    url = entry[1]
    next_page = True
    while next_page:
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        page_wrapper = soup.find('div', id='mw-pages')
        if not page_wrapper:
            next_page = False
            continue
        content_wrapper = page_wrapper.find('div', class_='mw-content-ltr')
        if not content_wrapper:
            next_page = False
            continue
        for movie in content_wrapper.find_all('a'):
            movie_dict = {}
            movie_dict['title'] = movie.get_text(strip=True)
            movie_dict['state'] = state_name
            movie_dict['wiki_link'] = f'https://en.wikipedia.org{movie["href"]}'
            movie_list.append(movie_dict)
        next_page_link = content_wrapper.find_previous_sibling('a')
        if next_page_link and (next_page_link.get_text(strip=True) == 'next page'):
            url = f"https://en.wikipedia.org{next_page_link['href']}"
        else:
            next_page = False
wiki_df = pd.DataFrame(movie_list)

('Alabama', 'https://en.wikipedia.org/wiki/Category:Films_set_in_Alabama')
('Alaska', 'https://en.wikipedia.org/wiki/Category:Films_set_in_Alaska')
('Alaska', 'https://en.wikipedia.org/wiki/Category:Balto_(film)')
('Alaska', 'https://en.wikipedia.org/wiki/Category:Films_based_on_The_Call_of_the_Wild')
('Alaska', 'https://en.wikipedia.org/wiki/Category:Northern_films')
('Alaska', 'https://en.wikipedia.org/wiki/Category:Mushing_films')
('Alaska', 'https://en.wikipedia.org/wiki/Category:Films_based_on_The_Spoilers_(Beach_novel)')
('Alaska', 'https://en.wikipedia.org/wiki/Category:Films_based_on_White_Fang')
('Arizona', 'https://en.wikipedia.org/wiki/Category:Films_set_in_Arizona')
('Arizona', 'https://en.wikipedia.org/wiki/Category:Films_set_in_Apache_County,_Arizona')
('Arizona', 'https://en.wikipedia.org/wiki/Category:Films_set_in_Blue_Gap,_Arizona')
('Arizona', 'https://en.wikipedia.org/wiki/Category:Films_set_in_Fort_Defiance,_Arizona')
('Arizona', 'https://en.wikipedia.org/wiki/Categ

('Illinois', 'https://en.wikipedia.org/wiki/Category:Documentary_films_about_Chicago')
('Illinois', 'https://en.wikipedia.org/wiki/Category:Films_about_the_Chicago_Outfit')
('Illinois', 'https://en.wikipedia.org/wiki/Category:Films_about_Al_Capone')
('Illinois', 'https://en.wikipedia.org/wiki/Category:Biographical_films_about_Al_Capone')
('Illinois', 'https://en.wikipedia.org/wiki/Category:Halloween_(franchise)_films')
('Indiana', 'https://en.wikipedia.org/wiki/Category:Films_set_in_Indiana')
('Indiana', 'https://en.wikipedia.org/wiki/Category:Films_set_in_Southwestern_Indiana')
('Indiana', 'https://en.wikipedia.org/wiki/Category:Films_set_in_Indianapolis')
('Iowa', 'https://en.wikipedia.org/wiki/Category:Films_set_in_Iowa')
('Kansas', 'https://en.wikipedia.org/wiki/Category:Films_set_in_Kansas')
('Kansas', 'https://en.wikipedia.org/wiki/Category:Films_based_on_The_Wizard_of_Oz')
('Kansas', 'https://en.wikipedia.org/wiki/Category:Animated_films_based_on_The_Wizard_of_Oz')
('Kansas', 'h

('Pennsylvania', 'https://en.wikipedia.org/wiki/Category:Rocky_characters')
('Pennsylvania', 'https://en.wikipedia.org/wiki/Category:Rocky_(film_series)_mass_media')
('Pennsylvania', 'https://en.wikipedia.org/wiki/Category:Rocky_(film_series)_films')
('Pennsylvania', 'https://en.wikipedia.org/wiki/Category:Rocky_music')
('Pennsylvania', 'https://en.wikipedia.org/wiki/Category:Songs_from_Rocky')
('Pennsylvania', 'https://en.wikipedia.org/wiki/Category:Rocky_soundtracks')
('Pennsylvania', 'https://en.wikipedia.org/wiki/Category:Rocky_video_games')
('Rhode Island', 'https://en.wikipedia.org/wiki/Category:Films_set_in_Rhode_Island')
('South Carolina', 'https://en.wikipedia.org/wiki/Category:Films_set_in_South_Carolina')
('South Carolina', 'https://en.wikipedia.org/wiki/Category:Films_set_in_Charleston,_South_Carolina')
('South Carolina', 'https://en.wikipedia.org/wiki/Category:Films_set_in_Rock_Hill,_South_Carolina')
('South Dakota', 'https://en.wikipedia.org/wiki/Category:Films_set_in_Sou

In [33]:
def get_imdb_link(url):
    print(url)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    try:
        ext_links = soup.find('span', id='External_links').find_parent().find_next_sibling('ul')
        return ext_links.find('a', text='IMDb').find_previous_sibling()['href']
    except:
        return 'n/a'
    return 'n/a'

In [30]:
wiki_df.drop_duplicates(inplace=True)
wiki_df['imdb_link'] = wiki_df['wiki_link'].map(get_imdb_link)

https://en.wikipedia.org/wiki/4_Little_Girls
https://en.wikipedia.org/wiki/Alabama_Moon_(film)
https://en.wikipedia.org/wiki/Another_Part_of_the_Forest_(film)
https://en.wikipedia.org/wiki/The_Barber_of_Birmingham
https://en.wikipedia.org/wiki/Before_I_Wake_(2016_film)
https://en.wikipedia.org/wiki/Big_Fish
https://en.wikipedia.org/wiki/Blue_Sky_(1994_film)
https://en.wikipedia.org/wiki/Body_Snatchers_(1993_film)
https://en.wikipedia.org/wiki/Borat
https://en.wikipedia.org/wiki/Bright_Road
https://en.wikipedia.org/wiki/Br%C3%BCno
https://en.wikipedia.org/wiki/Burning_Bright_(film)
https://en.wikipedia.org/wiki/The_Butler
https://en.wikipedia.org/wiki/ChromeSkull:_Laid_to_Rest_2
https://en.wikipedia.org/wiki/Constellation_(film)
https://en.wikipedia.org/wiki/Crazy_in_Alabama
https://en.wikipedia.org/wiki/Dead_Birds_(2004_film)
https://en.wikipedia.org/wiki/The_Death_of_Dick_Long
https://en.wikipedia.org/wiki/A_Dog_Named_Gucci
https://en.wikipedia.org/wiki/Due_Date
https://en.wikipedia.o

https://en.wikipedia.org/wiki/Runaway_Train_(film)
https://en.wikipedia.org/wiki/Salmonberries_(film)
https://en.wikipedia.org/wiki/The_Sea_Gypsies_(1978_film)
https://en.wikipedia.org/wiki/Seal_Island_(film)
https://en.wikipedia.org/wiki/The_Shooting_of_Dan_McGoo
https://en.wikipedia.org/wiki/The_Silver_Horde_(1930_film)
https://en.wikipedia.org/wiki/The_Silver_Horde_(1920_film)
https://en.wikipedia.org/wiki/The_Simpsons_Movie
https://en.wikipedia.org/wiki/Snow_Buddies
https://en.wikipedia.org/wiki/Snow_Dogs
https://en.wikipedia.org/wiki/Snow_Time
https://en.wikipedia.org/wiki/Spawn_of_the_North
https://en.wikipedia.org/wiki/Spirit_of_the_Wind
https://en.wikipedia.org/wiki/The_Spoilers_(1923_film)
https://en.wikipedia.org/wiki/The_Spoilers_(1930_film)
https://en.wikipedia.org/wiki/The_Spoilers_(1942_film)
https://en.wikipedia.org/wiki/The_Spoilers_(1955_film)
https://en.wikipedia.org/wiki/Stealth_(film)
https://en.wikipedia.org/wiki/Sugar_Mountain_(film)
https://en.wikipedia.org/wiki/

https://en.wikipedia.org/wiki/Okpik%27s_Dream
https://en.wikipedia.org/wiki/Snow_Dogs
https://en.wikipedia.org/wiki/Spirit_of_the_Wind
https://en.wikipedia.org/wiki/Stone_Fox
https://en.wikipedia.org/wiki/Togo_(film)
https://en.wikipedia.org/wiki/What_a_Nightmare,_Charlie_Brown!
https://en.wikipedia.org/wiki/White_Fang_(1991_film)
https://en.wikipedia.org/wiki/The_Spoilers_(1914_film)
https://en.wikipedia.org/wiki/The_Spoilers_(1923_film)
https://en.wikipedia.org/wiki/The_Spoilers_(1930_film)
https://en.wikipedia.org/wiki/The_Spoilers_(1942_film)
https://en.wikipedia.org/wiki/The_Spoilers_(1955_film)
https://en.wikipedia.org/wiki/Challenge_to_White_Fang
https://en.wikipedia.org/wiki/White_Fang_(1936_film)
https://en.wikipedia.org/wiki/White_Fang_(1973_film)
https://en.wikipedia.org/wiki/White_Fang_(1991_film)
https://en.wikipedia.org/wiki/White_Fang_2:_Myth_of_the_White_Wolf
https://en.wikipedia.org/wiki/White_Fang_to_the_Rescue
https://en.wikipedia.org/wiki/Zanna_Bianca_e_il_grande_Ki

https://en.wikipedia.org/wiki/Out_West_with_the_Hardys
https://en.wikipedia.org/wiki/Pete_%26_Cleo
https://en.wikipedia.org/wiki/Phase_IV_(1974_film)
https://en.wikipedia.org/wiki/The_Phoenix_Incident
https://en.wikipedia.org/wiki/Piranha_3D
https://en.wikipedia.org/wiki/Pocket_Money
https://en.wikipedia.org/wiki/Por_mis_pistolas_(1968_film)
https://en.wikipedia.org/wiki/Raise_Your_Voice
https://en.wikipedia.org/wiki/Raising_Arizona
https://en.wikipedia.org/wiki/Raising_Buchanan
https://en.wikipedia.org/wiki/Rambo_(2008_film)
https://en.wikipedia.org/wiki/Rambo:_Last_Blood
https://en.wikipedia.org/wiki/Riders_of_the_Purple_Sage_(1918_film)
https://en.wikipedia.org/wiki/Run,_Simon,_Run
https://en.wikipedia.org/wiki/Rustlers_(1949_film)
https://en.wikipedia.org/wiki/Savageland
https://en.wikipedia.org/wiki/The_Second_Time_Around_(1961_film)
https://en.wikipedia.org/wiki/Sicario_(2015_film)
https://en.wikipedia.org/wiki/South_of_Heaven,_West_of_Hell_(film)
https://en.wikipedia.org/wiki/Sp

https://en.wikipedia.org/wiki/The_Gospel_of_Eureka
https://en.wikipedia.org/wiki/I%27m_from_Arkansas
https://en.wikipedia.org/wiki/The_Kettles_in_the_Ozarks
https://en.wikipedia.org/wiki/The_Legend_of_Boggy_Creek
https://en.wikipedia.org/wiki/Massacre_at_Marble_City
https://en.wikipedia.org/wiki/Mississippi_Grind
https://en.wikipedia.org/wiki/Mud_(2012_film)
https://en.wikipedia.org/wiki/One_False_Move
https://en.wikipedia.org/wiki/Paradise_Lost_2:_Revelations
https://en.wikipedia.org/wiki/Paradise_Lost_3:_Purgatory
https://en.wikipedia.org/wiki/Paradise_Lost:_The_Child_Murders_at_Robin_Hood_Hills
https://en.wikipedia.org/wiki/Pass_the_Ammo
https://en.wikipedia.org/wiki/The_Pirates_of_the_Mississippi
https://en.wikipedia.org/wiki/Private_Life_(2018_film)
https://en.wikipedia.org/wiki/The_Revival_(film)
https://en.wikipedia.org/wiki/Rosalie_Goes_Shopping
https://en.wikipedia.org/wiki/September_30,_1955
https://en.wikipedia.org/wiki/Shotgun_Stories
https://en.wikipedia.org/wiki/Sling_Bla

...

https://en.wikipedia.org/wiki/Billy_Lynn%27s_Long_Halftime_Walk_(film)
https://en.wikipedia.org/wiki/The_Birth_of_a_Nation_(2016_film)
https://en.wikipedia.org/wiki/Black_Beauty_(1933_film)
https://en.wikipedia.org/wiki/Blue_Ruin
https://en.wikipedia.org/wiki/Body_of_Lies_(film)
https://en.wikipedia.org/wiki/The_Bourne_Identity_(2002_film)
https://en.wikipedia.org/wiki/The_Box_(2009_film)
https://en.wikipedia.org/wiki/The_Bride_Wore_Boots
https://en.wikipedia.org/wiki/Brother_Rat
https://en.wikipedia.org/wiki/Captain_John_Smith_and_Pocahontas
https://en.wikipedia.org/wiki/Chain_Reaction_(1996_film)
https://en.wikipedia.org/wiki/Cherry_Falls
https://en.wikipedia.org/wiki/A_Christmas_Kiss
https://en.wikipedia.org/wiki/Coming_Home_for_Christmas_(2017_film)
https://en.wikipedia.org/wiki/Country_Justice
https://en.wikipedia.org/wiki/The_Coward_(1915_film)
https://en.wikipedia.org/wiki/D.A.R.Y.L.
https://en.wikipedia.org/wiki/D.C._Sniper:_23_Days_of_Fear
https://en.wikipedia.org/wiki/Daniel_

https://en.wikipedia.org/wiki/Charlie_St._Cloud
https://en.wikipedia.org/wiki/Charlie,_the_Lonesome_Cougar
https://en.wikipedia.org/wiki/Christmas_with_Holly
https://en.wikipedia.org/wiki/Crimson_Tide_(film)
https://en.wikipedia.org/wiki/Dancer_in_the_Dark
https://en.wikipedia.org/wiki/Dante%27s_Peak
https://en.wikipedia.org/wiki/The_Dark_Divide
https://en.wikipedia.org/wiki/Dead_Man
https://en.wikipedia.org/wiki/Death_Do_Us_Part
https://en.wikipedia.org/wiki/Death_Note_(2017_film)
https://en.wikipedia.org/wiki/The_Details_(film)
https://en.wikipedia.org/wiki/Disclosure_(1994_film)
https://en.wikipedia.org/wiki/Distant_Thunder_(1988_film)
https://en.wikipedia.org/wiki/Disturbing_Behavior
https://en.wikipedia.org/wiki/Dog_Gone_(2008_film)
https://en.wikipedia.org/wiki/Double_Jeopardy_(1999_film)
https://en.wikipedia.org/wiki/The_Egg_and_I_(film)
https://en.wikipedia.org/wiki/Excess_Baggage_(1997_film)
https://en.wikipedia.org/wiki/Extraterrestrial_(2014_film)
https://en.wikipedia.org/wi

https://en.wikipedia.org/wiki/Fifty_Shades_Freed_(film)
https://en.wikipedia.org/wiki/Fifty_Shades_of_Grey_(film)
https://en.wikipedia.org/wiki/Finding_Mr._Right
https://en.wikipedia.org/wiki/Firewall_(film)
https://en.wikipedia.org/wiki/Get_Carter_(2000_film)
https://en.wikipedia.org/wiki/A_Guy_Thing
https://en.wikipedia.org/wiki/Halloween_III:_Season_of_the_Witch
https://en.wikipedia.org/wiki/The_Hand_That_Rocks_the_Cradle_(film)
https://en.wikipedia.org/wiki/Harry_and_the_Hendersons
https://en.wikipedia.org/wiki/Harry_in_Your_Pocket
https://en.wikipedia.org/wiki/The_Heart_of_the_Game
https://en.wikipedia.org/wiki/Highway_(2002_film)
https://en.wikipedia.org/wiki/House_of_Games
https://en.wikipedia.org/wiki/House_of_the_Dead_(film)
https://en.wikipedia.org/wiki/House_of_the_Dead_2_(film)
https://en.wikipedia.org/wiki/Humpday
https://en.wikipedia.org/wiki/The_Immaculate_Conception_of_Little_Dizzle
https://en.wikipedia.org/wiki/It_Happened_at_the_World%27s_Fair
https://en.wikipedia.org

https://en.wikipedia.org/wiki/The_Giant_Spider_Invasion
https://en.wikipedia.org/wiki/Give_Me_Liberty_(2019_film)
https://en.wikipedia.org/wiki/The_Godfather_of_Green_Bay
https://en.wikipedia.org/wiki/The_Great_Outdoors_(film)
https://en.wikipedia.org/wiki/Homer_(film)
https://en.wikipedia.org/wiki/In_the_Light_of_the_Moon
https://en.wikipedia.org/wiki/The_Incubus_(film)
https://en.wikipedia.org/wiki/Irresistible_(2020_film)
https://en.wikipedia.org/wiki/The_Lake_House_(film)
https://en.wikipedia.org/wiki/Lars_and_the_Real_Girl
https://en.wikipedia.org/wiki/Madison_County_(film)
https://en.wikipedia.org/wiki/A_Map_of_the_World_(film)
https://en.wikipedia.org/wiki/A_Merry_Friggin%27_Christmas
https://en.wikipedia.org/wiki/Michael_Clayton
https://en.wikipedia.org/wiki/Mister_Cory
https://en.wikipedia.org/wiki/Oconomowoc_(film)
https://en.wikipedia.org/wiki/Our_Vines_Have_Tender_Grapes
https://en.wikipedia.org/wiki/Outcast_(1937_film)
https://en.wikipedia.org/wiki/Public_Enemies_(2009_fil

,title,state,wiki_link,imdb_link
0,4 Little Girls,Alabama,https://en.wikipedia.org/wiki/4_Little_Girls,https://www.imdb.com/title/tt0118540/
1,Alabama Moon (film),Alabama,https://en.wikipedia.org/wiki/Alabama_Moon_(film),https://www.imdb.com/title/tt1300155/
2,Another Part of the Forest (film),Alabama,https://en.wikipedia.org/wiki/Another_Part_of_...,https://www.imdb.com/title/tt0040102/
3,The Barber of Birmingham,Alabama,https://en.wikipedia.org/wiki/The_Barber_of_Bi...,https://www.imdb.com/title/tt1844056/
4,Before I Wake (2016 film),Alabama,https://en.wikipedia.org/wiki/Before_I_Wake_(2...,https://www.imdb.com/title/tt3174376/
...,...,...,...,...
14864,Wild Horses (1985 film),Wyoming,https://en.wikipedia.org/wiki/Wild_Horses_(198...,https://www.imdb.com/title/tt0090324/
14865,Wind River (film),Wyoming,https://en.wikipedia.org/wiki/Wind_River_(film),https://www.imdb.com/title/tt5362988/
14866,Wyoming Outlaw,Wyoming,https://en.wikipedia.org/wiki/Wyoming_Outlaw,https://www.imdb.com/title/tt0032146/
14867,2012 (film),Wyoming,https://en.wikipedia.org/wiki/2012_(film),n/a


In [52]:
wiki_df = wiki_df[wiki_df['imdb_link']!='n/a']
wiki_df['title'] = wiki_df['title'].map(lambda x: x.split(' (')[0])
wiki_df['imdb_id'] = wiki_df['imdb_link'].map(lambda x: x[x.find('title/tt')+6:-1])
wiki_df.to_csv('Data/wiki_data.csv', index=False)

C:\Users\Andrew\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Andrew\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Andrew\anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea